In [59]:
import math


frames_output_dir_path = './calibration_frames'

In [1]:
import pathlib

import os
import sys
import time
import numpy as np
import cv2 as cv

In [61]:

target_frames_count = 10
frame_shot_delay_sec = 5
quit_key = 'q'

capture = cv.VideoCapture(0)

frame_count = 0
prev_frame_time = time.time()

while frame_count < target_frames_count:
    read_status, frame = capture.read()
    if not read_status:
        print('error: unable to read next frame, terminating...', file=sys.stderr)
        break
    
    curr_time = time.time()
    rem_sec_to_next_capture = max(0, frame_shot_delay_sec - (curr_time - prev_frame_time))
    
    if rem_sec_to_next_capture < 1e-6:
        frame_count += 1
        prev_frame_time = curr_time
        
        file_name = 'calib_frame_{:03d}.jpg'.format(frame_count)
        file_path = os.path.join(frames_output_dir_path, file_name)
        cv.imwrite(file_path, frame)
    
    text = '[{} / {}] time rem.: {:.4f} s.'.format(
        frame_count, target_frames_count, rem_sec_to_next_capture)
    cv.putText(frame, text, (10, 70), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv.LINE_AA)
    cv.imshow('Webcam feed (press \'{}\' to quit)'.format(quit_key), frame)
    
    if cv.waitKey(1) & 0xff == ord(quit_key):
        break

capture.release()
cv.destroyAllWindows()

In [164]:
def draw_countdown_clock(img, progress, radius=0.5, origin=None, alpha=0.5):
    radius = int(round((min(img.shape[:2]) / 2) * radius))

    if origin:
        center = origin
    else:
        cx, cy = img.shape[1] // 2, img.shape[0] // 2
        center = (cx, cy)

    angle = np.pi * (2 * progress - 0.5)

    fill_mask = np.ones_like(img, dtype=np.float32)
    cv.ellipse(
        fill_mask, center, (radius, radius), 0,
        270, np.degrees(angle), (alpha, alpha, alpha), thickness=-1,
        lineType=cv.LINE_AA
    )
    img = (img * fill_mask).round().astype(np.uint8)

    white = (255, 255, 255)
    cv.circle(img, center, radius, white, thickness=1, lineType=cv.LINE_AA)
    cv.circle(img, center, 3, white, thickness=-1, lineType=cv.LINE_AA)

    coords = np.asarray((np.cos(angle), np.sin(angle)))
    coords = ((coords * radius) + center).round().astype(np.int)
    cv.line(img, center, tuple(coords), white, thickness=1, lineType=cv.LINE_AA)

    return img


img = cv.imread("./calibration_frames/calib_frame_001.jpg")
img = draw_countdown_clock(img, 0.93)
cv.imshow("Preview", img)
cv.waitKey(0)
cv.destroyAllWindows()

In [144]:
np.degrees(-np.pi / 2)

-90.0

In [76]:
preview_captured = True

pattern_size = (9, 7)
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 100, 0.0001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(8,6,0)
objp = np.zeros((pattern_size[0] * pattern_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)

obj_points = [] # 3D points in real world space.
img_points = [] # 2D points in image plane.

for frame_file in pathlib.Path(frames_output_dir_path).iterdir():
    frame = cv.imread(str(frame_file))
    
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    ret, corners = cv.findChessboardCorners(gray, pattern_size)
    
    if not ret:
        continue
    
    obj_points.append(objp)

    corners_refined = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
    img_points.append(corners_refined)
    
    if preview_captured:
        time.sleep(1)
        
        frame = cv.drawChessboardCorners(frame, pattern_size, corners_refined, ret)
        cv.imshow('Captured calibration frame preview', frame)
        if cv.waitKey(1) & 0xff == ord('q'):
            break

cv.destroyAllWindows()

_, camera_matrix, dist_coefs, rvecs, tvecs = cv.calibrateCamera(
    obj_points, img_points, pattern_size, None, None
)

In [83]:
def draw_world_axes(img, rvec, tvec, K, dist_coefs=(0, 0, 0, 0), mag=3):
    def _draw_line(pt1, pt2, color):
        cv.arrowedLine(
            img, pt1, pt2, color=color, thickness=3, line_type=cv.LINE_AA,
            tipLength=0.2
        )

    points = np.float32(
        ((mag, 0, 0), (0, mag, 0), (0, 0, mag), (0, 0, 0))
    )
    points_projected, _ = cv.projectPoints(points, rvec, tvec, K, dist_coefs)
    origin = tuple(points_projected[3].ravel())

    _draw_line(origin, tuple(points_projected[0].ravel()), (0, 0, 255))
    _draw_line(origin, tuple(points_projected[1].ravel()), (0, 255, 0))
    _draw_line(origin, tuple(points_projected[2].ravel()), (255, 0, 0))

    return img

def draw_world_plane(
        img, rvec, tvec, K, dist_coefs=(0, 0, 0, 0), mag=5, alpha=0.7
):
    points = np.float32(
        ((-mag, -mag, 0), (mag, -mag, 0),
         (mag, mag, 0), (-mag, mag, 0))
    )
    points_projected, _ = cv.projectPoints(points, rvec, tvec, K, dist_coefs)
    points_projected = points_projected.round().astype(np.int32)

    img_overlay = np.ones_like(img, dtype=np.float32)
    cv.fillPoly(img_overlay, [points_projected], (alpha, alpha, alpha))
    img = (img * img_overlay).round().astype(np.uint8)

    return img

def img_show(img, winname="Preview"):
    cv.imshow(winname, img)
    cv.waitKey(0)
    cv.destroyWindow(winname)

img = cv.imread("./calibration_frames/calib_frame_001.jpg")
index = 0
img = draw_world_plane(img, rvecs[index], tvecs[index], camera_matrix, dist_coefs)
img = draw_world_axes(img, rvecs[index], tvecs[index], camera_matrix, dist_coefs)
img_show(img)

In [28]:
for frame_file in pathlib.Path(frames_output_dir_path).iterdir():
    frame = cv2.imread(str(frame_file))
    
    h, w = frame.shape[:2]
    
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(camera_matrix, dist_coefs, (w, h), 1, (w, h))
    dst = cv2.undistort(frame, camera_matrix, dist_coefs, None, newcameramtx)
    
    x, y, w, h = roi
    dst = dst[y:y+h, x:x+w]
    
    undist_file_path = pathlib.Path(frames_output_dir_path) / pathlib.Path('{}_undist{}'.format(frame_file.stem, frame_file.suffix))
    print('Saving... {}'.format(str(undist_file_path)))
    cv2.imwrite(str(undist_file_path), dst)

Saving... calibration_frames\calib_frame_001_undist.jpg
Saving... calibration_frames\calib_frame_002_undist.jpg
Saving... calibration_frames\calib_frame_003_undist.jpg
Saving... calibration_frames\calib_frame_004_undist.jpg
Saving... calibration_frames\calib_frame_005_undist.jpg
Saving... calibration_frames\calib_frame_006_undist.jpg
Saving... calibration_frames\calib_frame_007_undist.jpg
Saving... calibration_frames\calib_frame_008_undist.jpg
Saving... calibration_frames\calib_frame_009_undist.jpg
Saving... calibration_frames\calib_frame_010_undist.jpg


In [9]:
r1 = 640 / 480
r2 = 1280 / 720
r1, r2

1.3333333333333333